In [2]:
from fastai.tabular.all import *

In [3]:
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('/root/.fastai/data/adult_sample/models'),Path('/root/.fastai/data/adult_sample/adult.csv'),Path('/root/.fastai/data/adult_sample/export.pkl')]

In [4]:
df = pd.read_csv(path/'adult.csv')
df.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [5]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [Categorify, FillMissing, Normalize])

/usr/local/lib/python3.10/dist-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [6]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [7]:
to = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits)

/usr/local/lib/python3.10/dist-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [8]:
to.xs.iloc[:2]

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
5288,3,10,3,11,6,3,1,-0.926395,0.922959,1.144532
21354,5,12,5,2,5,3,1,0.029181,1.189559,-0.424105


In [9]:
dls = to.dataloaders(bs=64)

In [10]:
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Bachelors,Never-married,Other-service,Not-in-family,Asian-Pac-Islander,False,30.000000,182770.999652,13.0,<50k
1,Private,Bachelors,Divorced,Machine-op-inspct,Other-relative,White,False,43.000000,226902.000795,13.0,<50k
2,State-gov,Doctorate,Married-civ-spouse,Prof-specialty,Husband,Black,False,75.000001,113867.998920,16.0,>=50k
3,Self-emp-not-inc,Some-college,Married-civ-spouse,Farming-fishing,Husband,White,False,53.000000,279129.003045,10.0,<50k
4,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,False,37.000000,98776.001558,9.0,<50k
5,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,False,61.999999,98076.002891,13.0,>=50k
6,Private,Bachelors,Never-married,Prof-specialty,Own-child,White,False,25.000000,178842.999777,13.0,<50k
7,Private,Bachelors,Married-civ-spouse,#na#,Husband,White,True,42.000000,36271.005363,10.0,>=50k
8,Private,HS-grad,Never-married,Adm-clerical,Not-in-family,White,False,50.000000,104500.999395,9.0,<50k
9,Private,Some-college,Never-married,Exec-managerial,Not-in-family,White,False,28.000000,167335.999712,10.0,>=50k


In [11]:
learn = tabular_learner(dls, metrics=accuracy)

In [12]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.366768,0.363056,0.831235,00:04


In [13]:
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,5.0,1.0,3.0,0.0,1.0,5.0,2.0,0.470215,0.803768,-0.031946,0.0,1.0
1,5.0,12.0,3.0,8.0,1.0,5.0,1.0,0.470215,0.051846,-0.424105,1.0,0.0
2,5.0,13.0,5.0,11.0,4.0,5.0,1.0,-0.999900,-0.039485,1.536691,1.0,0.0
3,5.0,16.0,5.0,6.0,4.0,5.0,1.0,-0.999900,-1.207062,-0.031946,0.0,0.0
4,5.0,12.0,5.0,2.0,4.0,5.0,1.0,-1.146912,1.888129,-0.424105,0.0,0.0
5,5.0,16.0,5.0,13.0,4.0,5.0,1.0,-1.220418,0.739689,-0.031946,0.0,0.0
6,5.0,12.0,5.0,13.0,4.0,5.0,1.0,-1.073406,0.077549,-0.424105,0.0,0.0
7,5.0,1.0,5.0,7.0,3.0,5.0,1.0,-1.440935,0.891717,-1.600583,0.0,0.0
8,5.0,1.0,3.0,4.0,1.0,5.0,1.0,-0.705877,-0.529880,-1.600583,0.0,0.0


In [14]:
row, clas, probs = learn.predict(df.iloc[0])

/usr/local/lib/python3.10/dist-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [15]:
row.show()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Assoc-acdm,Married-civ-spouse,#na#,Wife,White,False,49.0,101320.001052,12.0,>=50k


In [16]:
clas, probs

(tensor(1), tensor([0.3675, 0.6325]))

In [17]:
test_df = df.copy()
test_df.drop(['salary'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

/usr/local/lib/python3.10/dist-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [18]:
to.xs[:3]

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
5288,3,10,3,11,6,3,1,-0.926395,0.922959,1.144532
21354,5,12,5,2,5,3,1,0.029181,1.189559,-0.424105
29561,5,12,3,13,1,3,1,1.646308,1.572993,-0.424105


In [19]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

In [20]:
X_test[:10]

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
5908,5,12,5,2,4,5,1,-1.293923,-0.139899,-0.424105
25294,5,12,1,4,5,5,1,-0.558866,0.513566,-0.424105
342,5,10,3,0,1,3,1,0.396709,3.167760,1.144532
7295,5,16,3,5,3,5,1,-0.632371,1.478546,-0.031946
28564,5,12,1,5,2,5,1,-0.191337,0.632233,-0.424105
6979,5,8,3,2,6,5,1,0.029181,0.124535,0.752372
30945,5,10,3,6,1,5,1,-0.411854,-0.127196,1.144532
20821,5,16,5,2,4,3,1,-1.220418,0.421655,-0.031946
23388,3,16,3,4,1,5,1,1.499296,-0.231867,-0.031946
22690,5,9,3,11,1,5,1,1.499296,-0.602692,0.360213
